In [1]:
import gensim
import nltk
import numpy as np
import pandas as pd
import keras
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import Dense, Embedding, Input, Activation
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
#x=train["comment_text"].values.tolist()
#train.str.replace('[^A-Za-z\s]+','')
#train.str.replace('\n','')

In [3]:
train['count_unique_word']=train['comment_text'].apply(lambda x: len(set(str(x).split())))
train['count_word']=train['comment_text'].apply(lambda x: len(str(x).split()))
train['word_unique_percent']=train['count_unique_word']*100/train['count_word']
train = train[train['word_unique_percent']>10]

In [4]:
x=train['comment_text'].values.tolist()
corpus= [nltk.word_tokenize(sent) for sent in x]
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("na").values

In [5]:
max_features=25000
maxlen=100
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x)
word_index=tokenizer.word_index
list_tokenized_train = tokenizer.texts_to_sequences(x)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [6]:
#train word embedding using words in training set
from gensim.models import Word2Vec
embedding=gensim.models.Word2Vec(corpus,size=128)
embedding.save("word2vec.model")
model=Word2Vec.load('word2vec.model')

In [7]:
#Map vocabularies and trained word vector to a dictionary file
my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    my_dict[key] = model.wv[key]

In [8]:
#load weight matrix from our pretrained word2vec model
max_features = 25000 
maxlen = 100
embed_size=128
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(0, 1, (nb_words, embed_size))
for word,i in word_index.items():
    if i >= max_features: continue
    embedding_vector=my_dict.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [10]:
inp = Input(shape=(maxlen,))
NN = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
NN = Bidirectional(LSTM(50, return_sequences=True))(NN)
NN = GlobalMaxPool1D()(NN)
NN = Dropout(0.1)(NN)
NN = Dense(50,activation = 'relu' )(NN)
NN = Dropout(0.1)(NN)
NN = Dense(6, activation ='sigmoid')(NN)
x = Model(inputs=inp, outputs=NN)
x.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
x.fit(X_t, y, epochs=3, batch_size=100, validation_split=0.2, verbose=0)

In [13]:
y_test = x.predict([X_te], batch_size=1024, verbose=1)

153164/153164 [==============================] - 185s 1ms/step


In [21]:
#sample_submission1 = pd.read_csv("sample_submission1.csv")
#sample_submission1[list_classes] = y_test
#sample_submission1.to_csv('submission.csv', index=False)